# ISOLATION FOREST FOR ANOMALY DETECTION

Algoritmos de Isolation Forest para detecção de Anolamia na Folha de Pagamento
Importação do Modulo de Pré-processamento

In [42]:
import sys
sys.path.append("C:/Users/joaoc/Documents/MT/pagamento_servidores/src")
import process_servants as ps
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import IsolationForest
import matplotlib.pyplot as plt
from statsmodels.tsa.arima.model import ARIMA
from sklearn.metrics import mean_absolute_error
import pandas as pd
from janitor import clean_names
from datetime import datetime


## Função para pre-processamento dos dados
Utiliza-se funções importadas do arquivo process_servants.py, no qual foram pré-definidas algumas funções para o pré-processamento dos dados

In [5]:
def process_registration_data_rendim(file_path):
    """Process registration data from Excel file"""
    
    df = (
        pd.read_excel(file_path)
        .pipe(clean_names)
        .pipe(ps.convert_rendim_to_numeric)
        .pipe(ps.process_month_column)
        .pipe(ps.padronizar_ferias)
        .pipe(ps.padronizar_decimo_terceiro)
        .pipe(ps.somar_colunas)
        )
    df = df.drop(df[df['rubrica'] == "S/rubrica"].index)
    df = df.groupby(['cpf_servidor', 'mes', 'rubrica'])['rendim'].sum().reset_index()
    df = df.pivot(index = ['cpf_servidor', 'mes'], columns='rubrica', values='rendim').reset_index()

    return df

## Função para Leitura dos Dados

In [7]:
def processed_data(brute_data_file_path): 

    registration_df = process_registration_data_rendim(brute_data_file_path)
    
    return registration_df

Importação da Base de Dados

In [9]:
brute_file_path = 'C:/Users/joaoc/Documents/MT/pagamento_servidores/data/raw/servidores_mes_rubricas.xlsx'

Limpeza e Pre-processamento dos dados para tentar identificar o caso a seguir como um Anomaly Detection
![image.png](attachment:1cf54f27-e734-4093-a92f-766002dc7ed0.png)

In [11]:
registration_df = processed_data(brute_file_path, "dados_12_meses_cadastro_e_std")

C:\Users\joaoc\anaconda3\Lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


Processed data shapes:
Registration data: (37918, 249)


## Isolation Forest - Rubricas Detalhadas

In [22]:
cpf_list = registration_df['cpf_servidor'].unique()

In [24]:
resultado_rubricas = []

In [99]:
resultado_rubricas = []

for each in cpf_list:
    dados = registration_df[registration_df['cpf_servidor'] == each]
    dados = dados.sort_values(by='mes', ascending=False)
    dados = dados.fillna(0)
    series = dados.drop('cpf_servidor', axis=1)
    series = series.loc[:, (series != 0).any(axis=0)]
    series = series.set_index('mes')
    window = series
    print(each)
    
    while len(window) > 10 :
        # Define contaminação de forma mais eficiente
        contamination = 1/len(series) if len(series) < 20 else 1/20

        # Cria e ajusta o modelo
        iso_forest = IsolationForest(
            contamination=contamination,
            random_state=42,
            n_jobs=-1  # Paraleliza o processamento
        )
        
        # Faz a predição
        anomalias = iso_forest.fit_predict(window)
        
        # Ou crie a coluna se necessário:
        if 'DECIMO TERCEIRO' not in window.columns:
            window['DECIMO TERCEIRO'] = 0  # Valor padrão ou cálculo
        
        # Agora execute a verificação de anomalias
        if anomalias[0] == -1 and window.loc[window.index[0], 'DECIMO TERCEIRO'] == 0:
            resultado_rubricas.append({
                "cpf_servidor": dados['cpf_servidor'].iloc[0],
                "mes": window.index[0]
            })
        print(window.index[0])
        if window.index[0] < datetime(2023,8,31):
            break
        # Exclui o ultimo mes, para verificar anomalia nos meses anteriores
        window = window.iloc[1:]



4362160
11092190
13956701
2025-03-01 00:00:00
2025-02-01 00:00:00
2025-01-01 00:00:00
2024-12-01 00:00:00
2024-11-01 00:00:00
2024-10-01 00:00:00
2024-09-01 00:00:00
2024-08-01 00:00:00
2024-07-01 00:00:00
2024-06-01 00:00:00
2024-05-01 00:00:00
2024-04-01 00:00:00
2024-03-01 00:00:00
2024-02-01 00:00:00
2024-01-01 00:00:00
2023-12-01 00:00:00
2023-11-01 00:00:00
30052165
2025-03-01 00:00:00
2025-02-01 00:00:00
2025-01-01 00:00:00
2024-12-01 00:00:00
2024-11-01 00:00:00
2024-10-01 00:00:00
2024-09-01 00:00:00
2024-08-01 00:00:00
2024-07-01 00:00:00
2024-06-01 00:00:00
2024-05-01 00:00:00
2024-04-01 00:00:00
2024-03-01 00:00:00
2024-02-01 00:00:00
2024-01-01 00:00:00
2023-12-01 00:00:00
2023-11-01 00:00:00
31035795
2025-03-01 00:00:00
2025-02-01 00:00:00
2025-01-01 00:00:00
2024-12-01 00:00:00
2024-11-01 00:00:00
2024-10-01 00:00:00
2024-09-01 00:00:00
2024-08-01 00:00:00
2024-07-01 00:00:00
2024-06-01 00:00:00
2024-05-01 00:00:00
2024-04-01 00:00:00
2024-03-01 00:00:00
2024-02-01 00:00

KeyboardInterrupt: 

In [101]:
resultado_rubricas

[{'cpf_servidor': 13956701, 'mes': Timestamp('2024-10-01 00:00:00')},
 {'cpf_servidor': 13956701, 'mes': Timestamp('2024-05-01 00:00:00')},
 {'cpf_servidor': 43345158, 'mes': Timestamp('2024-05-01 00:00:00')},
 {'cpf_servidor': 45237174, 'mes': Timestamp('2024-09-01 00:00:00')},
 {'cpf_servidor': 128470119, 'mes': Timestamp('2024-05-01 00:00:00')},
 {'cpf_servidor': 129423750, 'mes': Timestamp('2024-05-01 00:00:00')},
 {'cpf_servidor': 145183173, 'mes': Timestamp('2024-10-01 00:00:00')},
 {'cpf_servidor': 145183173, 'mes': Timestamp('2024-05-01 00:00:00')},
 {'cpf_servidor': 358035767, 'mes': Timestamp('2024-05-01 00:00:00')},
 {'cpf_servidor': 409600130, 'mes': Timestamp('2024-05-01 00:00:00')},
 {'cpf_servidor': 409600130, 'mes': Timestamp('2024-01-01 00:00:00')},
 {'cpf_servidor': 425287122, 'mes': Timestamp('2024-09-01 00:00:00')},
 {'cpf_servidor': 425287122, 'mes': Timestamp('2023-12-01 00:00:00')},
 {'cpf_servidor': 459115170, 'mes': Timestamp('2024-05-01 00:00:00')}]

## Isolation Forest - Remuneracao Total

In [ ]:
resultado_remun = []

series_remun = dados.drop('cpf_servidor', axis=1)
series_remun = series_remun.loc[:, (series_remun != 0).any(axis=0)]
series_remun['redim'] = series_remun.iloc[:, 1:].sum(axis=1)
series_remun = series_remun.set_index('mes')
series_remun = series_remun.iloc[:, -1:]


In [ ]:
window = series_remun
while len(window) > 10:
    # Define contaminação de forma mais eficiente
    contamination = 1/len(series) if len(series) < 20 else 1/20
    
    # Cria e ajusta o modelo
    iso_forest = IsolationForest(
        contamination=contamination,
        random_state=42,
        n_jobs=-1  # Paraleliza o processamento
    )
    
    # Faz a predição
    anomalias = iso_forest.fit_predict(window)
    
    # Verifica anomalias de forma vetorizada
    if anomalias[-1] == -1:  # Verifica apenas a última observação
        resultado_remun.append({
            "cpf_servidor": dados['cpf_servidor'].iloc[0],
            "mes": window.index[-1]
        })
        
    # Exclui o ultimo mes, para verificar anomalia nos meses anteriores
    window = window.iloc[:-1, :]


In [ ]:
resultado_remun